In [1]:
import cv2
import mediapipe as mp
import numpy as np

In [2]:
def euclidean_distance(x1, x2):
    total = 0
    for i in range(1, len(x1)):
        total += (x1[i]-x2[i])**2
    return total ** 0.5

In [3]:
def stateDistance(vector):
    if len(vector) == 3:
        a1 = euclidean_distance(vector[0], vector[1])
        a2 = euclidean_distance(vector[1], vector[2])
        return [a1, a2]
    if len(vector) == 4:
        a1 = euclidean_distance(vector[0], vector[1])
        a2 = euclidean_distance(vector[1], vector[2])
        a3 = euclidean_distance(vector[2], vector[3])
        return [a1, a2, a3]

In [50]:
class HandDetector:
    def __init__(self, mode=False, maxHands=1, modelC=1, detectionCon=0.5, trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.modelC = modelC
        self.trackCon = trackCon

        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands, self.modelC, self.detectionCon, self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils

    def findHands(self, img):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)

        if self.results.multi_hand_landmarks:
            for handLandmarks in self.results.multi_hand_landmarks:
                self.mpDraw.draw_landmarks(img, handLandmarks, self.mpHands.HAND_CONNECTIONS, self.mpDraw.DrawingSpec(color=(255,0,0), thickness=2, circle_radius=2), self.mpDraw.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2))

        return img


    def findPosition(self, img):
        landmarkList = []

        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[0]
            
            for id, lm in enumerate(myHand.landmark):
                h,w,c = img.shape
                cx, cy = int(lm.x*w,), int(lm.y*h)
                landmarkList.append([id, cx,cy])
        return landmarkList

    def naturalGrasps_gestureRecognizer(self, landmarkList):
        thumbState = ""
        indexState = ""
        middleState = ""
        ringState = ""
        pinkyState = ""
        handGrasps = ""
        
        if len(landmarkList) != 0:
            if landmarkList[2][1] > landmarkList[3][1] > landmarkList[4][1]:
                thumbState = "CLOSE"
            elif landmarkList[2][1] < landmarkList[3][1] < landmarkList[4][1]:
                thumbState = "OPEN"
                 
            if landmarkList[6][2] > landmarkList[7][2] > landmarkList[8][2]:
                indexState = "OPEN"
            elif landmarkList[6][2] < landmarkList[7][2] < landmarkList[8][2]:
                indexState = "CLOSE"
            elif landmarkList[6][2] > landmarkList[7][2] and landmarkList[8][2] > landmarkList[7][2]:
                indexState = "HALF_OPEN"
            
            if landmarkList[10][2] > landmarkList[11][2] > landmarkList[12][2]:
                middleState = "OPEN"
            elif landmarkList[10][2] < landmarkList[11][2] < landmarkList[12][2]:
                middleState = "CLOSE"
            elif landmarkList[10][2] > landmarkList[11][2] and landmarkList[12][2] > landmarkList[11][2]:
                middleState = "HALF_OPEN"
                
            if landmarkList[14][2] > landmarkList[15][2] > landmarkList[16][2]:
                ringState = "OPEN"
            elif landmarkList[14][2] < landmarkList[15][2] < landmarkList[16][2]:
                ringState = "CLOSE"
            elif landmarkList[14][2] > landmarkList[15][2] and landmarkList[16][2] > landmarkList[15][2]:
                ringState = "HALF_OPEN"
                
            if landmarkList[18][2] > landmarkList[19][2] > landmarkList[20][2]:
                pinkyState = "OPEN"
            elif landmarkList[18][2] < landmarkList[19][2] < landmarkList[20][2]:
                pinkyState = "CLOSE"
            elif landmarkList[18][2] > landmarkList[19][2] and landmarkList[20][2] > landmarkList[19][2]:
                pinkyState = "HALF_OPEN"

        # print( "thumbState : ", thumbState , " indexState : ", indexState, " middleState : ", middleState, " ringState : ", ringState, " pinkyState : ", pinkyState)

            





        if indexState in ['CLOSE',""] and middleState in ['CLOSE',""] and ringState in ['CLOSE',""] and pinkyState in ['CLOSE',""]:
            di_p = euclidean_distance(landmarkList[6], landmarkList[5])
            dm_p = euclidean_distance(landmarkList[9], landmarkList[10])
            dr_p = euclidean_distance(landmarkList[13], landmarkList[14])
            dp_p = euclidean_distance(landmarkList[17], landmarkList[18])
            d_t_i = euclidean_distance(landmarkList[4], landmarkList[8])
            dt_m = euclidean_distance(landmarkList[4], landmarkList[12])

            d_r_p = euclidean_distance(landmarkList[16], landmarkList[20])

            t_distal_phalanx = round(euclidean_distance(landmarkList[4], landmarkList[3]), 4)
            d_w_t_cmtcp = euclidean_distance(landmarkList[0], landmarkList[1])
            d_t_tip_i_distal_itphlg_joint = euclidean_distance(landmarkList[4], landmarkList[6])

            d_w_t = euclidean_distance(landmarkList[0], landmarkList[4])
            d_w_i_prox = euclidean_distance(landmarkList[0], landmarkList[6])
            d_w_m_prox = euclidean_distance(landmarkList[0], landmarkList[10])
            d_w_r_prox = euclidean_distance(landmarkList[0], landmarkList[14])
            d_w_p_prox = euclidean_distance(landmarkList[0], landmarkList[18])

            d_w_i = euclidean_distance(landmarkList[0], landmarkList[8])
            d_w_m = euclidean_distance(landmarkList[0], landmarkList[12])
            d_w_r = euclidean_distance(landmarkList[0], landmarkList[16])
            d_w_p = euclidean_distance(landmarkList[0], landmarkList[20])
            
            
            # 2 Small Diameter
            if di_p < 0.7*t_distal_phalanx and dm_p < 0.7*t_distal_phalanx and dr_p < 0.7*t_distal_phalanx and dp_p < 0.7*t_distal_phalanx and (d_w_i_prox > d_w_t and d_w_m_prox > d_w_t and d_w_r_prox > d_w_t and d_w_p_prox > d_w_t):
                handGrasps = 'small_diameter'

            # 3 Medium Wrap
            elif 0.7*t_distal_phalanx < di_p and 0.7*t_distal_phalanx < dm_p and 0.7*t_distal_phalanx < dr_p and 0.6*t_distal_phalanx < dp_p and d_t_i < 1.2*t_distal_phalanx and dt_m > 1.3*t_distal_phalanx and d_r_p < 1.3*t_distal_phalanx:
                handGrasps = 'medium_wrap'

            # 23 Lareral Tripod
            elif di_p > 0.7*t_distal_phalanx and dm_p < 0.7*t_distal_phalanx and dr_p < 0.7*t_distal_phalanx and dp_p < 0.7*t_distal_phalanx and d_t_i < t_distal_phalanx :
                handGrasps = 'lateral_tripod'
        

            # 16 lateral
            elif indexState in ["CLOSE", ""] and middleState in ["CLOSE", ""] and ringState in ["CLOSE", ""] and pinkyState in ["CLOSE", ""] and (d_w_i_prox < d_w_t and d_w_m_prox < d_w_t and d_w_r_prox < d_w_t and d_w_p_prox < d_w_t) and (d_t_tip_i_distal_itphlg_joint < d_w_t_cmtcp) and (d_w_i < 2.7*d_w_t_cmtcp and d_w_m < 2.7*d_w_t_cmtcp and d_w_r < 2.7*d_w_t_cmtcp and d_w_p < 2.7*d_w_t_cmtcp):
                    handGrasps = "lateral"
                

        # 10 Power Disk
        if indexState == 'HALF_OPEN' and middleState == "HALF_OPEN" and ringState == "HALF_OPEN" and pinkyState == "HALF_OPEN":
            d_t_i = euclidean_distance(landmarkList[4], landmarkList[8])
            d_t_m = euclidean_distance(landmarkList[4], landmarkList[12])
            d_t_r = euclidean_distance(landmarkList[4], landmarkList[16])
            d_t_p = euclidean_distance(landmarkList[4], landmarkList[20])
            d_w_t_cmtcp = euclidean_distance(landmarkList[0], landmarkList[1]) # wrist to thumb carpometacarpal joint

            if d_t_i > 2 * d_w_t_cmtcp and  d_t_m > 1.2*d_w_t_cmtcp  and d_t_r > 1.2*d_w_t_cmtcp and  d_t_p > 1.2*d_w_t_cmtcp:
                handGrasps = 'power_disk'



        


        
        
        if middleState == "CLOSE" and ringState == 'CLOSE' and pinkyState == "CLOSE":
            dt_i_p = round(euclidean_distance(landmarkList[5], landmarkList[8]), 4)
            dt_m_p = round(euclidean_distance(landmarkList[9], landmarkList[12]), 4)
            dt_r_p = round(euclidean_distance(landmarkList[13], landmarkList[16]), 4)
            dt_p_p = round(euclidean_distance(landmarkList[17], landmarkList[20]), 4)


            d_w_i = round(euclidean_distance(landmarkList[0], landmarkList[8]), 4)
            d_w_m = round(euclidean_distance(landmarkList[0], landmarkList[12]), 4)
            d_w_r = round(euclidean_distance(landmarkList[0], landmarkList[16]), 4)
            d_w_p = round(euclidean_distance(landmarkList[0], landmarkList[20]), 4)
            
            d_t_tip_i_mtcp = round(euclidean_distance(landmarkList[4], landmarkList[5]), 4)

            t_metacarpal = round(euclidean_distance(landmarkList[0], landmarkList[1]), 4)
            t_distal_phalanx = round(euclidean_distance(landmarkList[4], landmarkList[3]), 4)

            d_t_i = euclidean_distance(landmarkList[4], landmarkList[8])
            d_w_t_cmtcp = euclidean_distance(landmarkList[0], landmarkList[1]) # wrist to thumb carpometacarpal joint
            d_i_tip_m_distal_itphlg_joint = euclidean_distance(landmarkList[6], landmarkList[10])
            

            if indexState == 'CLOSE' and dt_i_p < 0.8 * t_distal_phalanx and dt_m_p < 0.8 * t_distal_phalanx and dt_r_p <  0.8 * t_distal_phalanx and dt_p_p < 0.8 * t_distal_phalanx:
                    
                    # 5 light tool
                    if 1.5*t_distal_phalanx <= d_t_tip_i_mtcp <= 2.5 * t_distal_phalanx: 
                        handGrasps = "light_tool"

                    # 15 fixed hook
                    if 2.5 * t_distal_phalanx < d_t_tip_i_mtcp : 
                        handGrasps = "fixed_hook"

            
            # 4 adducted thumb
            elif indexState in ['HALF_OPEN', "OPEN"] and d_w_t_cmtcp < d_t_i < 2 * d_w_t_cmtcp and d_w_p < 2*d_w_t_cmtcp and d_i_tip_m_distal_itphlg_joint < d_w_t_cmtcp:
                    handGrasps = 'adducted_thumb'


            # 28 Palmar
            elif d_w_i < 1.7 * t_metacarpal and d_w_m < 1.7 * t_metacarpal and d_w_r < 1.7 * t_metacarpal and d_w_p < 1.7 * t_metacarpal:
                handGrasps = "palmar"

        
        if indexState == 'OPEN' and middleState == "OPEN" and ringState == 'OPEN' and pinkyState == "OPEN":

            d_i_m = euclidean_distance(landmarkList[8], landmarkList[12])
            d_m_r = euclidean_distance(landmarkList[12], landmarkList[16])
            d_r_p = euclidean_distance(landmarkList[16], landmarkList[20])
            d_t_tip_i_distal_itphlg_joint = euclidean_distance(landmarkList[4], landmarkList[6])
            d_t_tip_p_mtcp = euclidean_distance(landmarkList[4], landmarkList[17])
            d_w_t_cmtcp = euclidean_distance(landmarkList[0], landmarkList[1])

            l_i = euclidean_distance(landmarkList[8], landmarkList[6])
            l_m = euclidean_distance(landmarkList[12], landmarkList[10])
            l_r = euclidean_distance(landmarkList[16], landmarkList[14])
            l_p = euclidean_distance(landmarkList[20], landmarkList[18])

            d_t_tip_i_meta = euclidean_distance(landmarkList[4], landmarkList[5])

            # 20 parallel Extension
            if d_i_m < 0.7*d_w_t_cmtcp and d_m_r < 0.7*d_w_t_cmtcp and d_r_p < 0.7*d_w_t_cmtcp and d_t_tip_i_distal_itphlg_joint < d_w_t_cmtcp:
                handGrasps = "parallel_extension"

           
            # hand is contracted
            if l_i < 1.2*d_w_t_cmtcp and l_m < 1.2*d_w_t_cmtcp and l_r < 1.2*d_w_t_cmtcp and l_p < 1.2*d_w_t_cmtcp:

                # 12 precision disk
                if d_t_tip_i_meta < 1.2*d_w_t_cmtcp and d_t_tip_i_distal_itphlg_joint < 1.2*d_w_t_cmtcp:
                    handGrasps = "precision_disk"

                # 1 large diameter
                else:  
                    handGrasps = "large_diameter"





        if pinkyState == "CLOSE":

            d_i_m = euclidean_distance(landmarkList[8], landmarkList[12])
            d_t_i = euclidean_distance(landmarkList[4], landmarkList[8])
            d_t_m = euclidean_distance(landmarkList[4], landmarkList[12])
            d_t_r = euclidean_distance(landmarkList[4], landmarkList[16])
            d_m_r = euclidean_distance(landmarkList[12], landmarkList[16])
            
            d_r_p = euclidean_distance(landmarkList[16], landmarkList[20])



            d_w_t = euclidean_distance(landmarkList[0], landmarkList[4])
            d_w_i = euclidean_distance(landmarkList[0], landmarkList[8])
            d_w_m = euclidean_distance(landmarkList[0], landmarkList[12])
            d_w_r = euclidean_distance(landmarkList[0], landmarkList[16])
            d_w_p = euclidean_distance(landmarkList[0], landmarkList[20])

            d_w_t_cmtcp = euclidean_distance(landmarkList[0], landmarkList[1])
            t_distal_phalanx = euclidean_distance(landmarkList[4], landmarkList[3])
            t_distal_i_meta = euclidean_distance(landmarkList[3], landmarkList[5])

            d_tip_i_metacarpal = euclidean_distance(landmarkList[4], landmarkList[5])

            l_r_prox = euclidean_distance(landmarkList[13], landmarkList[14])


            print(d_m_r > d_w_t_cmtcp)



            # 24 sphere 4 finger
            if indexState != "OPEN" and middleState != "OPEN" and ringState != "OPEN" and d_t_i > 1.5*d_w_t_cmtcp and d_t_m > 1.5*d_w_t_cmtcp and d_t_r > 1.5*d_w_t_cmtcp and d_m_r < 2*d_w_t_cmtcp and l_r_prox > 0.5*d_w_t_cmtcp and d_w_i > 2*d_w_t_cmtcp and d_w_m > 2*d_w_t_cmtcp and d_tip_i_metacarpal < 1.5 * d_w_t_cmtcp:
                handGrasps = "sphere_4_fingers"

            # 7 prismatic 3 finger 
            if d_t_i < 0.75*d_w_t_cmtcp and d_t_m < 0.75*d_w_t_cmtcp and d_m_r < 0.8*d_w_t_cmtcp and d_r_p > 0.6*d_w_t_cmtcp and l_r_prox > d_w_t_cmtcp:
                    handGrasps = "prismatic_3_finger"

            # 25 quadpod
            elif 0.75* d_w_t_cmtcp <= d_t_i < 1.5*d_w_t_cmtcp and 0.75*d_w_t_cmtcp <= d_t_m < 1.5*d_w_t_cmtcp and 0.75*d_w_t_cmtcp <= d_t_r < 1.5*d_w_t_cmtcp:
                handGrasps = "quadpod"


            
        


        
        if ringState == "CLOSE" and pinkyState == "CLOSE":
            d_i_m = euclidean_distance(landmarkList[8], landmarkList[12])

            d_t_i = euclidean_distance(landmarkList[4], landmarkList[8])
            d_t_m = euclidean_distance(landmarkList[4], landmarkList[12])
            d_m_r = euclidean_distance(landmarkList[12], landmarkList[16])
            l_r_prox = euclidean_distance(landmarkList[13], landmarkList[14])


            d_w_t = euclidean_distance(landmarkList[0], landmarkList[4])
            d_w_i = euclidean_distance(landmarkList[0], landmarkList[8])
            d_w_m = euclidean_distance(landmarkList[0], landmarkList[12])
            d_w_r = euclidean_distance(landmarkList[0], landmarkList[16])
            d_w_p = euclidean_distance(landmarkList[0], landmarkList[20])

            d_w_t_cmtcp = euclidean_distance(landmarkList[0], landmarkList[1])
            t_distal_phalanx = euclidean_distance(landmarkList[4], landmarkList[3])
            t_distal_i_meta = euclidean_distance(landmarkList[3], landmarkList[5])



            #21 adduction grip
            if indexState == "OPEN" and middleState == "OPEN":
                if d_i_m < 2 * t_distal_phalanx and t_distal_i_meta < 1.5 * t_distal_phalanx and d_t_m > 1.5*t_distal_phalanx: 
                    handGrasps = "adduction_grip"

            else:

                

                if d_w_r < 2*d_w_t_cmtcp: 
            
                    # prismatic_2_finger
                    if d_t_i < 0.75*d_w_t_cmtcp and d_t_m < 0.85*d_w_t_cmtcp and ((d_w_i > d_w_r and d_w_i > d_w_p) and (d_w_m > d_w_r and d_w_m > d_w_p) and (d_m_r > 0.5*d_w_t_cmtcp) and (d_w_t < d_w_i and d_w_t < d_w_m)):
                        handGrasps = "prismatic_2_finger"

                    # tripod
                    elif 0.75* d_w_t_cmtcp <= d_t_i < 1.5*d_w_t_cmtcp and 0.75*d_w_t_cmtcp <= d_t_m < 1.5*d_w_t_cmtcp and d_m_r > 1.5*d_w_t_cmtcp:
                        handGrasps = "tripod"

                    # sphere 3 fingers
                    elif d_t_i > 1.2*d_w_t_cmtcp and d_t_m > 1.5*d_w_t_cmtcp and d_w_i > 2*d_w_t_cmtcp and d_w_m > 2*d_w_t_cmtcp and (0.85 * d_w_i < d_w_m < 1.15 * d_w_i):
                        handGrasps = "sphere_3_fingers"






        # 6 prismatic 4 finger 
        if indexState != 'OPEN' and middleState != 'OPEN' and ringState != 'OPEN' and pinkyState != "OPEN":
            d_t_i = euclidean_distance(landmarkList[4], landmarkList[8])
            d_t_m = euclidean_distance(landmarkList[4], landmarkList[12])
            d_i_m = euclidean_distance(landmarkList[8], landmarkList[12])
            d_m_r = euclidean_distance(landmarkList[12], landmarkList[16])
            d_r_p = euclidean_distance(landmarkList[16], landmarkList[20])

            l_m_prox = euclidean_distance(landmarkList[9], landmarkList[10])
            l_r_prox = euclidean_distance(landmarkList[13], landmarkList[14])



            d_w_t_cmtcp = euclidean_distance(landmarkList[0], landmarkList[1]) # wrist to thumb carpometacarpal joint

            if d_t_i < d_w_t_cmtcp and d_i_m < d_w_t_cmtcp and d_m_r < d_w_t_cmtcp and d_r_p < d_w_t_cmtcp and d_t_m <= d_t_i and (l_r_prox > d_w_t_cmtcp) and (l_m_prox > d_w_t_cmtcp):
                handGrasps = "prismatic_4_finger"

            










        

        # 9 palmar pinch
        if middleState == "OPEN" and ringState == 'OPEN' and pinkyState == "OPEN":
            di_t = euclidean_distance(landmarkList[4], landmarkList[8])
            t_distal_phalanx = round(euclidean_distance(landmarkList[4], landmarkList[3]), 4)

            l_i = euclidean_distance(landmarkList[8], landmarkList[5])
            l_m = euclidean_distance(landmarkList[9], landmarkList[12])
            l_r = euclidean_distance(landmarkList[13], landmarkList[16])
            l_p = euclidean_distance(landmarkList[17], landmarkList[20])

            if di_t < t_distal_phalanx:
                if l_m > l_r > l_p:
                    handGrasps = "palmar_pinch"
                elif l_i < l_m < l_r:
                    handGrasps = "tip_pinch"


        

      

        
        if (thumbState in ['OPEN', ""]) and indexState == 'OPEN' and middleState == "CLOSE" and ringState == 'CLOSE' and pinkyState == "CLOSE":
            d_t_tip_i_distal_itphlg_joint = euclidean_distance(landmarkList[4], landmarkList[6])
            d_i_tip_m_distal_itphlg_joint = euclidean_distance(landmarkList[8], landmarkList[10])
            t_distal_phalanx = euclidean_distance(landmarkList[4], landmarkList[3])
            d_i_m = euclidean_distance(landmarkList[8], landmarkList[12])
            d_t_i = euclidean_distance(landmarkList[4], landmarkList[8])
            d_w_t_cmtcp = euclidean_distance(landmarkList[0], landmarkList[1])



            # 17 index finger extension
            if d_t_tip_i_distal_itphlg_joint > 1.3*t_distal_phalanx  and d_i_tip_m_distal_itphlg_joint > 1.5 * d_w_t_cmtcp and ringState=="CLOSE" and  d_i_m > 2*t_distal_phalanx:
                handGrasps = "index_finger_extension"


            # 30 ventral
            if (d_t_tip_i_distal_itphlg_joint < 1.2* t_distal_phalanx and d_i_m > 3*t_distal_phalanx and d_t_i > 2*t_distal_phalanx) and d_i_tip_m_distal_itphlg_joint > 1.5 * d_w_t_cmtcp:
                handGrasps = "ventral"

        
        if middleState in ['CLOSE', ""] and ringState in ['CLOSE', ""] and pinkyState in ['CLOSE', ""]:
            # 27 stick
            d_t_i = euclidean_distance(landmarkList[4], landmarkList[8])
            t_distal_phalanx = euclidean_distance(landmarkList[4], landmarkList[3])
            d_i_m = euclidean_distance(landmarkList[8], landmarkList[12])

            if (d_t_i < t_distal_phalanx and d_i_m > 2.2*t_distal_phalanx):
                handGrasps = "stick"
            # elif d_i_m < 2*t_distal_phalanx and d_t_i < t_distal_phalanx:
            #     handGrasps = "tip_stick"
        





            
        # 22 tip pinch
        if pinkyState == "OPEN" and ringState == "OPEN":
            l_m = euclidean_distance(landmarkList[9], landmarkList[12])
            l_r = euclidean_distance(landmarkList[13], landmarkList[16])
            l_p = euclidean_distance(landmarkList[17], landmarkList[20])

        
        if middleState == "OPEN" and ringState == 'OPEN' and pinkyState == "OPEN":
            di_t = euclidean_distance(landmarkList[4], landmarkList[8])
            di_m = euclidean_distance(landmarkList[8], landmarkList[12])
            d_w_t_cmtcp = euclidean_distance(landmarkList[0], landmarkList[1]) # wrist to thumb carpometacarpal joint

            l_r = euclidean_distance(landmarkList[13], landmarkList[16])
            l_m = euclidean_distance(landmarkList[9], landmarkList[12])

            if d_w_t_cmtcp < di_t < 2*d_w_t_cmtcp and di_m > d_w_t_cmtcp :

                # 29 ring
                if l_m > l_r:
                    handGrasps = "ring"

                # 31 Inferior Pincer
                else:
                    handGrasps = "inferior_pincer"

        return handGrasps

In [52]:
if __name__ == "__main__":
    cap = cv2.VideoCapture(0)
    detector = HandDetector()

    natural_gesture_dic = {'large_diameter':1, 'small_diameter':2, 'medium_wrap':3, 'tip_pinch':4, 'light_tool':5, 'prismatic_4_fingers':6, 'prismatic_3_fingers':7, 'prismatic_2_fingers':8, 'palmar_pinch':9, 'power_disk':10,
                          'power_sphere':11, 'precision_disk':12, 'precision_sphere':13, 'tripod':14, 'fixed_hook':15, 'lateral':16, 'index_finger_extension':17, 'extension_type':18, 'writing_tripod':19, 'parallel_extension':20,
                          'lateral_tripod':21, 'tripod_variation':22, 'palmar':23, 'sphere_4_fingers':24, 'quadpod':25, 'sphere_3_fingers':26, 'stick':27, 'ventral':28, 'ring':29, 'inferior_pinch':30}
    
    gest_val = []
    while True:
        success, img = cap.read()
        img = detector.findHands(img)
        landmarkList = detector.findPosition(img)

        #print(landmarkList)

        if len(landmarkList) != 0:
            
            hg = detector.naturalGrasps_gestureRecognizer(landmarkList)
            cv2.putText(img, detector.naturalGrasps_gestureRecognizer(landmarkList), (landmarkList[0][1]-50, landmarkList[0][2]-300), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255),2)
            
            if hg in natural_gesture_dic:
                gest_val.append(natural_gesture_dic[hg])
        #print(gest_val)
    
        cv2.imshow("Hand Tracking", img)
        
        if cv2.waitKey(1) == ord('q'):
            break
    
    cv2.destroyAllWindows()
    cap.release()

I0000 00:00:1731228794.720716       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 76.3), renderer: Apple M1


thumbState :    indexState :  OPEN  middleState :  OPEN  ringState :  OPEN  pinkyState :  OPEN
thumbState :    indexState :  OPEN  middleState :  OPEN  ringState :  OPEN  pinkyState :  OPEN
thumbState :  OPEN  indexState :  OPEN  middleState :  OPEN  ringState :  OPEN  pinkyState :  OPEN
thumbState :  OPEN  indexState :  OPEN  middleState :  OPEN  ringState :  OPEN  pinkyState :  OPEN
thumbState :  OPEN  indexState :  OPEN  middleState :  OPEN  ringState :  OPEN  pinkyState :  OPEN
thumbState :  OPEN  indexState :  OPEN  middleState :  OPEN  ringState :  OPEN  pinkyState :  OPEN
thumbState :  OPEN  indexState :  OPEN  middleState :  OPEN  ringState :  OPEN  pinkyState :  OPEN
thumbState :  OPEN  indexState :  OPEN  middleState :  OPEN  ringState :  OPEN  pinkyState :  OPEN
thumbState :  OPEN  indexState :  OPEN  middleState :  OPEN  ringState :  OPEN  pinkyState :  OPEN
thumbState :  OPEN  indexState :  OPEN  middleState :  OPEN  ringState :  OPEN  pinkyState :  OPEN
thumbState :  OPEN